# Simple Naive Bayes Classification

Version: 1.0 (January 2018)

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
import numpy as np
import pandas as pd

In [ ]:
%run Tutorial1_Appendix.ipynb #Fetch some functions from the Appendix, such as get_genome_pair(student_id)
my_genomes = get_genome_pair("B00672276")
print(my_genomes) # Print the genome IDs out as text below

In [ ]:
with open("tutorial1_appendix_data/" + my_genomes[0] + "_subset.faa", 'r') as genome_file:
    print(genome_file.readline())
    
with open("tutorial1_appendix_data/" + my_genomes[1] + "_subset.faa", 'r') as genome_file:
    print(genome_file.readline())

In [ ]:
def count_words(sequence, w):
    count_dict = {}
    for i in range(0, len(sequence) - w + 1):
        word = sequence[i:i+w]
        if word not in count_dict:
            count_dict[word] = 1
        else:
            count_dict[word] += 1
    return count_dict

def index_fasta_file(fasta_filename, w = 4):
    word_dict = {} # Stores each of the protein data points
    sequence = "" # Start building the sequence, since it can be spread across multiple lines
    row = 0 # Keep track of which sequence we have processed
    with open(fasta_filename, 'r') as fasta_file:
        for line in fasta_file: # Loop through the file line-by-line
            if line[0] == ">": # Every time it hits a label, we are done processing the last sequence
                if len(sequence) >= w:
                    # Count the words using the function above
                    count_dict = count_words(sequence, w)
                    # For each word, we want to add it to our matrix that we are building
                    for word, count in count_dict.items():
                        # If we've seen the word before, append to the existing count list
                        if word in word_dict:
                            word_dict[word] = word_dict[word].append(
                                                              pd.Series([count],index=[row])
                                                                    )
                        # If it's a new word, start a counting list
                        else:
                            word_dict[word] = pd.Series([count],index=[row])
                    row += 1
                sequence = ""
            else: # We have a protein sequence, not a label
                sequence += line.strip() # Build the full sequence string, removing trailing whitespace with strip() 
    return pd.DataFrame(word_dict).fillna(0) # Return the counts as a DataFrame

In [ ]:
word_size = 1 # Word size: Change this to change the word length for each genome

count_matrix_1 = index_fasta_file("tutorial1_appendix_data/" + my_genomes[0] + "_subset.faa", word_size)
count_matrix_2 = index_fasta_file("tutorial1_appendix_data/" + my_genomes[1] + "_subset.faa", word_size)

genome_labels = [0]*count_matrix_1.shape[0] + [1]*count_matrix_2.shape[0]

In [ ]:
print(count_matrix_1)
print(count_matrix_2)
combined_matrix = count_matrix_1.append(count_matrix_2).fillna(0)

In [ ]:
def NB(X, y):
    normalized_X = normalize(X, axis=1)

    X_train, X_test, y_train, y_test = train_test_split(normalized_X, y, test_size = 0.33)
    
    nb = GaussianNB()
    nb.fit(X_train, y_train)
    train_score = nb.score(X_train, y_train)
    test_score = nb.score(X_test, y_test)

    print("Training set score: %.3f" % (train_score,))
    print("Test set score: %.3f" % (test_score, ))

In [ ]:
NB(combined_matrix, genome_labels)